# 渠道模型 (Canal Model)

`渠道 (Canal)` 类代表一个具有梯形横截面的渠道。它使用曼宁方程来模拟水流。本Jupyter Notebook演示了如何使用 `Canal` 类来模拟一个渠道段的水动力学过程。

## 状态变量

渠道的状态由以下变量定义：

- `volume` (float): 渠道中当前的水量 (m³)。
- `water_level` (float): 渠道当前的水位 (m)。
- `outflow` (float): 当前时间步计算出的渠道出流量 (m³/s)。

## 参数

渠道的物理特性由这些参数定义：

- `bottom_width` (float): 渠道底部的宽度 (m)。
- `length` (float): 渠道段的长度 (m)。
- `slope` (float): 渠道河床的纵向坡度 (无量纲)。
- `side_slope_z` (float): 渠道边坡的坡度 (z:1 中的 z，水平:垂直)。
- `manning_n` (float): 曼宁糙率系数。

## 仿真示例

以下代码设置并运行一个渠道的仿真。它用一个初始水量和物理参数来初始化渠道，然后在一段时间内施加一个恒定的入流。仿真结果，包括水量、水位和出流量，将被绘制成图表。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` (`pip install matplotlib`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.canal import Canal
from swp.core.interfaces import State

# 定义渠道的初始状态和参数
initial_state = State(volume=1000.0, water_level=0.0, outflow=0.0)
parameters = {
    'bottom_width': 10.0,    # 米
    'length': 1000.0,        # 米
    'slope': 0.001,          # 无量纲
    'side_slope_z': 2.0,     # z:1, 水平:垂直
    'manning_n': 0.03        # 曼宁糙率系数
}

# 创建一个Canal模型的实例
canal = Canal(name="main_canal", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 60  # 时间步长（秒）(1分钟)
simulation_duration = 3600 * 2  # 2小时
num_steps = int(simulation_duration / dt)
inflow_rate = 5.0  # 恒定入流 (m³/s)

# 存储结果用于绘图
history = []

# 运行仿真循环
for _ in range(num_steps):
    canal.set_inflow(inflow_rate)
    current_state = canal.step(action=None, dt=dt)
    history.append(current_state.copy()) # 在每一步存储状态的副本

# 打印渠道的最终状态
print("最终状态:")
print(canal.get_state())

# 准备绘图数据
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
volumes = [s['volume'] for s in history]
water_levels = [s['water_level'] for s in history]
outflows = [s['outflow'] for s in history]

# 绘制结果
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

ax1.plot(time_in_hours, volumes, label='水量 (Volume)')
ax1.set_ylabel('水量 (m³)')
ax1.set_title('渠道动态仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_hours, water_levels, label='水位 (Water Level)', color='orange')
ax2.set_ylabel('水位 (m)')
ax2.grid(True)
ax2.legend()

ax3.plot(time_in_hours, outflows, label='出流量 (Outflow)', color='green')
ax3.set_xlabel('时间 (小时)')
ax3.set_ylabel('出流量 (m³/s)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()
